# Замикання

В Python функції можуть бути оголошені всередині інших функцій. У такому випадку, змінні, оголошені в тілі зовнішьної функції, будуть доступні внутрішнім функціям.

In [25]:
def closure_func(y):
    a = 1
    x = 2
    def inner_func():
        print(f"This i an a param: {a}")
    inner_func()

Це стосується, також, і аргументів функції:

In [5]:
def yet_another_closure(*args):
    def inner_func():
        return sum(args)
    return inner_func()

In [6]:
yet_another_closure(20, 19, 20)

59

### Замиканням (closure) inner_func ми назвемо набір змінних, доступних їй через те, що вони оголошені у зовнішній функції (closure_func або yet_another_closure). 

Область видимості, котру утворює замикання, називається не-локальною (nonlocal).

# Декоратори

Декоратор - це спосіб "обгорнути", доповнити код певної функції.

In [27]:
from typing import Callable, TypeVar, Any

T = TypeVar("T")

def my_precious_decorator(func: Callable[T, T]) -> Callable[T, T]:
    def wrap() -> T:
        print("side effect yo!")
        func()
        print("another side effect yo!")
    return wrap

In [30]:
decorated_function = my_precious_decorator(printer)

In [28]:
@my_precious_decorator
def printer():
    print("some stuff")

In [29]:
printer()

side effect yo!
some stuff
another side effect yo!


Так само, ви можете обгортати функції з аргументами

In [66]:
from functools import wraps

def yet_another_precious_decorator(func: Callable[T, T]) -> Callable[T, T]:
    @wraps(func)
    def wrap(*args, **kwargs) -> T:
        print("Here we go again")
        a = func(*args, **kwargs)
        print("We did our calculations")
        return a
    return wrap

In [67]:
def sum_all_vars(a, b, *args) -> float:
    logger.info("Was called")
    return a + b + sum(args)

In [68]:
@yet_another_precious_decorator
def sum_all_vars_decorated(a, b, *args):
    return a + b + sum(args)

In [69]:
sum_all_vars_decorated(1, 2)

Here we go again
We did our calculations


3

In [71]:
sum_all_vars_decorated.__wrapped__(1, 2, 3, 4, 5,)

15

In [41]:
def sum_all_vars_not_decorated(a, b, *args):
    return a + b + sum(args)

In [43]:
sum_all_vars_not_decorated(1, 2)

3

In [39]:
sum_all_vars(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

Here we go again
We did our calculations


55

І передавати аргументи в декоратор, або створювати змінні в декораторі.

In [20]:
from functools import wraps

def multiplicative_decorator(multiplier: int):
    and_another_multiplier: int = 3
    def inside_func(func):
        @wraps(func)
        def wrap(*args, **kwargs):
            print(f"This result will be altered my multiplying on {and_another_multiplier} and {multiplier}")
            result = func(*args, **kwargs)
            print(f"Initial result {result}")
            return result*multiplier*and_another_multiplier
        return wrap
    return inside_func

In [21]:
@multiplicative_decorator(num_of_tries = 20)
def reverse_and_sum(*args):
    return 1/sum(args)

# Корисні декоратори

Декоратори - хліб з маслом Python-програміста. Якщо ви їх не пишете, то ви їх точно будете постійно використовувати. 

Декілька найбільш потрібних декораторів (частину з них ви дізнаєтесь пізніше в рамках курсу з ООП):

* @lru_cache
* @property
* @wraps
* @abstractmethod
* @staticmethod
* @dataclass
* @jit
* @app.get в FastAPI

І так далі.

Гарний список Python-декораторів є [тут](https://github.com/lord63/awesome-python-decorator)

In [65]:
from functools import lru_cache

In [66]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n*factorial(n-1)

In [67]:
%%timeit

factorial(15)

2.2 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [68]:
@lru_cache
def factorial_cached(n):
    if n == 1:
        return 1
    else:
        return n*factorial(n-1)

In [69]:
%%timeit

factorial_cached(15)

82.7 ns ± 5.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [75]:
from functools import wraps

def my_cache(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        call = (func, args)
        if call in container.keys():
            a = container[call]
            return a
        else:
            a = func(*args, **kwargs)
            container[call] = a
            return a
    return wrapper

In [77]:
my_cache(fib)

<function __main__.fib(n)>

In [71]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [61]:
%%timeit

fib(10)
fib(10)

617 ns ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [62]:
%%timeit

fib.__wrapped__(10)
fib.__wrapped__(10)

1.65 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [49]:
%%timeit

def factorial_not_cached(n):
    if n == 1:
        return 1
    else:
        return n*factorial_not_cached(n-1)

factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)
factorial_not_cached(6)

6.75 µs ± 718 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
